# Toronto's Neighborhood Analysis

In this notebook I'll be analising the clusters related to Toronto's negihborhood.

In [9]:
#let's import everything needed.
import pandas as pd
import numpy as np
import folium
from bs4 import BeautifulSoup
import geocoder
import requests

So, first let's get the data in the wikipedia website.

In [32]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

data = dfs[0]
data

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,NaN
176,M6Z,Not assigned,NaN
177,M7Z,Not assigned,NaN
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...
